In [38]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import col, when, lit
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from nltk.stem.snowball import SnowballStemmer
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.sql.types import StringType, ArrayType

In [39]:
df = sqlContext.read.option('header', True).option('delimiter', ',').csv(
    'hdfs://localhost:9000/project/cleaneddata.csv')
df.show()

+-----+--------------------+--------------------+
|   id|               title|                tags|
+-----+--------------------+--------------------+
|  862|           Toy Story|led by woodi andy...|
| 8844|             Jumanji|when sibl judi an...|
|15602|    Grumpier Old Men|a famili wed reig...|
|31357|   Waiting to Exhale|"cheat on mistrea...|
|11862|Father of the Bri...|just when georg b...|
|  949|                Heat|obsess master thi...|
|11860|             Sabrina|an ugli duckl hav...|
|45325|        Tom and Huck|a mischiev young ...|
| 9091|        Sudden Death|intern action sup...|
|  710|           GoldenEye|jame bond must un...|
| 9087|The American Pres...|widow us presid a...|
|12110|Dracula: Dead and...|when a lawyer sho...|
|21032|               Balto|an outcast half-w...|
|10858|               Nixon|an all-star cast ...|
| 1408|    Cutthroat Island|morgan adam and h...|
|  524|              Casino|the life of the g...|
| 4584|Sense and Sensibi...|rich mr dashwood ...|


In [11]:
# Tokenize text
tokenizer = Tokenizer(inputCol='tags', outputCol='words_token')
df_words_token = tokenizer.transform(df).select('id','title', 'words_token')


In [40]:
df_words_token.show()

+-----+--------------------+--------------------+
|   id|               title|         words_token|
+-----+--------------------+--------------------+
|  862|           Toy Story|[led, by, woodi, ...|
| 8844|             Jumanji|[when, sibl, judi...|
|15602|    Grumpier Old Men|[a, famili, wed, ...|
|31357|   Waiting to Exhale|["cheat, on, mist...|
|11862|Father of the Bri...|[just, when, geor...|
|  949|                Heat|[obsess, master, ...|
|11860|             Sabrina|[an, ugli, duckl,...|
|45325|        Tom and Huck|[a, mischiev, you...|
| 9091|        Sudden Death|[intern, action, ...|
|  710|           GoldenEye|[jame, bond, must...|
| 9087|The American Pres...|[widow, us, presi...|
|12110|Dracula: Dead and...|[when, a, lawyer,...|
|21032|               Balto|[an, outcast, hal...|
|10858|               Nixon|[an, all-star, ca...|
| 1408|    Cutthroat Island|[morgan, adam, an...|
|  524|              Casino|[the, life, of, t...|
| 4584|Sense and Sensibi...|[rich, mr, dashwo...|


In [41]:
# Remove stop words
remover = StopWordsRemover(inputCol='words_token', outputCol='tags_clean')
df_words_no_stopw = remover.transform(df_words_token).select('id','title', 'tags_clean')


In [49]:
df_words_no_stopw.show()

+-----+--------------------+--------------------+
|   id|               title|          tags_clean|
+-----+--------------------+--------------------+
|  862|           Toy Story|[led, woodi, andy...|
| 8844|             Jumanji|[sibl, judi, pete...|
|15602|    Grumpier Old Men|[famili, wed, rei...|
|31357|   Waiting to Exhale|["cheat, mistreat...|
|11862|Father of the Bri...|[georg, bank, ha,...|
|  949|                Heat|[obsess, master, ...|
|11860|             Sabrina|[ugli, duckl, und...|
|45325|        Tom and Huck|[mischiev, young,...|
| 9091|        Sudden Death|[intern, action, ...|
|  710|           GoldenEye|[jame, bond, must...|
| 9087|The American Pres...|[widow, us, presi...|
|12110|Dracula: Dead and...|[lawyer, show, va...|
|21032|               Balto|[outcast, half-wo...|
|10858|               Nixon|[all-star, cast, ...|
| 1408|    Cutthroat Island|[morgan, adam, sl...|
|  524|              Casino|[life, gambl, par...|
| 4584|Sense and Sensibi...|[rich, mr, dashwo...|


In [52]:
from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(vectorSize=5, seed=42, inputCol="tags_clean", outputCol="tags_vec")

In [53]:
word2Vec.setMaxIter(10)

Word2Vec_7cdea03c4fad

In [54]:
word2Vec.getMaxIter()

10

In [55]:
word2Vec.clear(word2Vec.maxIter)
model = word2Vec.fit(df_words_no_stopw)
model.getMinCount()

5

In [56]:
model.setInputCol("tags_clean")

Word2VecModel: uid=Word2Vec_7cdea03c4fad, numWords=25119, vectorSize=5

In [57]:
model.getVectors().show()

+-------------------+--------------------+
|               word|              vector|
+-------------------+--------------------+
|         bingcrosbi|[-0.8500708937644...|
|           leemajor|[-0.2095074802637...|
|           clarissa|[0.00553934602066...|
|           incident|[0.04493460059165...|
|      meteorologist|[0.02840667031705...|
|      eugenepallett|[-0.1949429959058...|
|            serious|[0.01164876297116...|
|        francesbarb|[-0.1995777338743...|
|              brink|[0.08022177219390...|
|        misadventur|[0.26905563473701...|
|               9/11|[0.23603622615337...|
|          foolproof|[0.07430215924978...|
|            miracul|[0.10923843085765...|
|    corinnaharfouch|[-0.2538802325725...|
|michaelclarkeduncan|[-0.3469690978527...|
|            mafioso|[0.04059444367885...|
|             coach'|[-0.0025814063847...|
|              manli|[0.06812771409749...|
|          forgotten|[0.10097167640924...|
|             layton|[0.06701664626598...|
+----------

In [64]:
model.getVectors().collect()

[Row(word='bingcrosbi', vector=DenseVector([-0.8501, -0.0626, 1.0239, -0.4931, -0.2003])),
 Row(word='leemajor', vector=DenseVector([-0.2095, -0.0414, 0.5821, -0.2998, -0.0529])),
 Row(word='clarissa', vector=DenseVector([0.0055, 0.0586, -0.082, -0.0315, -0.106])),
 Row(word='incident', vector=DenseVector([0.0449, 0.1035, -0.0253, 0.0341, -0.0246])),
 Row(word='meteorologist', vector=DenseVector([0.0284, 0.2029, -0.1022, -0.001, -0.0756])),
 Row(word='eugenepallett', vector=DenseVector([-0.1949, -0.1404, 0.1608, -0.0008, -0.1365])),
 Row(word='serious', vector=DenseVector([0.0116, 0.1715, -0.5207, 0.0191, 0.1673])),
 Row(word='francesbarb', vector=DenseVector([-0.1996, -0.2083, 0.43, -0.0867, -0.0024])),
 Row(word='brink', vector=DenseVector([0.0802, 0.3965, -0.1888, 0.0307, 0.0658])),
 Row(word='misadventur', vector=DenseVector([0.2691, 0.2421, -0.1964, -0.1567, 0.1121])),
 Row(word='9/11', vector=DenseVector([0.236, 0.1816, 0.1197, 0.0891, 0.044])),
 Row(word='foolproof', vector=Dens

In [59]:
df_words_no_stopw.show()

+-----+--------------------+--------------------+
|   id|               title|          tags_clean|
+-----+--------------------+--------------------+
|  862|           Toy Story|[led, woodi, andy...|
| 8844|             Jumanji|[sibl, judi, pete...|
|15602|    Grumpier Old Men|[famili, wed, rei...|
|31357|   Waiting to Exhale|["cheat, mistreat...|
|11862|Father of the Bri...|[georg, bank, ha,...|
|  949|                Heat|[obsess, master, ...|
|11860|             Sabrina|[ugli, duckl, und...|
|45325|        Tom and Huck|[mischiev, young,...|
| 9091|        Sudden Death|[intern, action, ...|
|  710|           GoldenEye|[jame, bond, must...|
| 9087|The American Pres...|[widow, us, presi...|
|12110|Dracula: Dead and...|[lawyer, show, va...|
|21032|               Balto|[outcast, half-wo...|
|10858|               Nixon|[all-star, cast, ...|
| 1408|    Cutthroat Island|[morgan, adam, sl...|
|  524|              Casino|[life, gambl, par...|
| 4584|Sense and Sensibi...|[rich, mr, dashwo...|


In [23]:
# # Stem text
# stemmer = SnowballStemmer(language='english')
# stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
# df_stemmed = df_words_no_stopw.withColumn("words_stemmed", stemmer_udf("tags_clean")).select('id','title', 'words_stemmed')
# df_stemmed.show()

In [25]:
# from pyspark.ml.feature import CountVectorizer
# cv = CountVectorizer(inputCol="tags_clean", outputCol="tag_vec")
# model = cv.fit(df_words_no_stopw)
# result = model.transform(df_words_no_stopw)
# result.show(truncate=False)

In [43]:
# from pyspark.ml.feature import HashingTF, IDF
# hashingTF = HashingTF(inputCol="tags_clean", outputCol="tag_hash")
# tf = hashingTF.transform(df_words_no_stopw)

In [44]:
# idf = IDF(inputCol="tag_hash", outputCol="feature").fit(tf)
# tfidf = idf.transform(tf)

In [45]:
# tf.show()

+-----+--------------------+--------------------+--------------------+
|   id|               title|          tags_clean|            tag_hash|
+-----+--------------------+--------------------+--------------------+
|  862|           Toy Story|[led, woodi, andy...|(262144,[15756,20...|
| 8844|             Jumanji|[sibl, judi, pete...|(262144,[3329,680...|
|15602|    Grumpier Old Men|[famili, wed, rei...|(262144,[9129,188...|
|31357|   Waiting to Exhale|["cheat, mistreat...|(262144,[921,5765...|
|11862|Father of the Bri...|[georg, bank, ha,...|(262144,[1493,545...|
|  949|                Heat|[obsess, master, ...|(262144,[3924,651...|
|11860|             Sabrina|[ugli, duckl, und...|(262144,[951,6998...|
|45325|        Tom and Huck|[mischiev, young,...|(262144,[6512,786...|
| 9091|        Sudden Death|[intern, action, ...|(262144,[1876,110...|
|  710|           GoldenEye|[jame, bond, must...|(262144,[1791,169...|
| 9087|The American Pres...|[widow, us, presi...|(262144,[6512,907...|
|12110

In [32]:
# tfidf.show()

+-----+--------------------+--------------------+--------------------+--------------------+
|   id|               title|          tags_clean|            tag_hash|             feature|
+-----+--------------------+--------------------+--------------------+--------------------+
|  862|           Toy Story|[led, woodi, andy...|(262144,[15756,20...|(262144,[15756,20...|
| 8844|             Jumanji|[sibl, judi, pete...|(262144,[3329,680...|(262144,[3329,680...|
|15602|    Grumpier Old Men|[famili, wed, rei...|(262144,[9129,188...|(262144,[9129,188...|
|31357|   Waiting to Exhale|["cheat, mistreat...|(262144,[921,5765...|(262144,[921,5765...|
|11862|Father of the Bri...|[georg, bank, ha,...|(262144,[1493,545...|(262144,[1493,545...|
|  949|                Heat|[obsess, master, ...|(262144,[3924,651...|(262144,[3924,651...|
|11860|             Sabrina|[ugli, duckl, und...|(262144,[951,6998...|(262144,[951,6998...|
|45325|        Tom and Huck|[mischiev, young,...|(262144,[6512,786...|(262144,[6

In [46]:
# from pyspark.ml.feature import Normalizer
# normalizer = Normalizer(inputCol="feature", outputCol="norm")
# data = normalizer.transform(tfidf)

In [47]:
# data.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|   id|               title|          tags_clean|            tag_hash|             feature|                norm|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  862|           Toy Story|[led, woodi, andy...|(262144,[15756,20...|(262144,[15756,20...|(262144,[15756,20...|
| 8844|             Jumanji|[sibl, judi, pete...|(262144,[3329,680...|(262144,[3329,680...|(262144,[3329,680...|
|15602|    Grumpier Old Men|[famili, wed, rei...|(262144,[9129,188...|(262144,[9129,188...|(262144,[9129,188...|
|31357|   Waiting to Exhale|["cheat, mistreat...|(262144,[921,5765...|(262144,[921,5765...|(262144,[921,5765...|
|11862|Father of the Bri...|[georg, bank, ha,...|(262144,[1493,545...|(262144,[1493,545...|(262144,[1493,545...|
|  949|                Heat|[obsess, master, ...|(262144,[3924,651...|(262144,[3924,651...|(2621

In [48]:
# from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
# mat = IndexedRowMatrix(
#     data.select("norm").rdd.map(lambda row: IndexedRow(row.norm.toArray()))).toBlockMatrix()
# dot = mat.multiply(mat.transpose())
# dot.toLocalMatrix().toArray()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 1 times, most recent failure: Lost task 0.0 in stage 25.0 (TID 48) (192.168.0.100 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ckkhandare/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/ckkhandare/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/ckkhandare/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/ckkhandare/spark-3.1.2-bin-hadoop3.2/python/pyspark/rdd.py", line 1560, in takeUpToNumLeft
    yield next(iterator)
  File "/home/ckkhandare/spark-3.1.2-b